# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.122481e+02     1.580378e+00
 * time: 0.6927509307861328
     1     1.080469e+01     8.655927e-01
 * time: 2.0174529552459717
     2    -1.212781e+01     1.057845e+00
 * time: 2.672315835952759
     3    -3.390737e+01     7.453569e-01
 * time: 3.6613059043884277
     4    -4.746127e+01     5.398960e-01
 * time: 4.613008975982666
     5    -5.675641e+01     2.276962e-01
 * time: 5.5925610065460205
     6    -5.973211e+01     1.146884e-01
 * time: 6.2305169105529785
     7    -6.085408e+01     4.534887e-02
 * time: 6.8802008628845215
     8    -6.126638e+01     8.407974e-02
 * time: 7.53013801574707
     9    -6.158256e+01     3.277903e-02
 * time: 8.198517799377441
    10    -6.179460e+01     2.544870e-02
 * time: 8.841769933700562
    11    -6.195054e+01     1.846255e-02
 * time: 9.48632287979126
    12    -6.200957e+01     1.846941e-02
 * time: 10.12818694114685
    13    -6.207505e+01     1.510530e-02
 * time: 10.777406930923462
  

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671075
    AtomicLocal         -18.8557707
    AtomicNonlocal      14.8522661
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485383 
    Xc                  -19.3336824

    total               -62.261666453288
